# Creating a Zone-Budget with FEFLOW

This is an example of how to create a waterbalance of a model that is subdivided into multiple subdomains.

*Requirements: Note that **this example makes use of [ifm_contrib](https://github.com/dhi/ifm_contrib)**. Make sure to install the latest version before using this code.*


In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ifm_contrib as ifm
ifm.getKernelVersion()

7207

## Pre-Flight Checks

**Subdomains are Elemental Selections**

The subdomains of the model must be defined in the model als elemental selections.
Make sure that the selections are non-overlapping. 
All selections must have a common prefix, defined by the user. We use `subdomain<n>` as names for subdomains in this example.

**Run the model**

This example works on a dac file and uses a steady-state example to keep things simple.
It can be easily adapted to a transient model, please let us know if you need assistance.

**Do necessary settings**

Please adapt the below variables to your needs:

In [2]:
## test model
file_results = "./results/subdomain_test1.dac"  # path to dac-file
force_cache_update = True  # Set this to false to avoid a recomputation of the border nodes.
selection_prefix = "subdomain"  # the prefix that identifies 

## Pre-Processing

**Convert Elemental Selections to Node lists**

Load the model and convert the elemental selections to node lists. As the implementation in ifm_contrib is not very efficition, the result is cached to save time when repeating this step. Set `force_cache_update = True` to force an update if you have made changes to the selections, set `force_cache_update = False` to save computation time for consecutive runs.

In [3]:
# load the model and get all relevant zones
doc = ifm.loadDocument(file_results)
zones = [s for s in doc.c.sel.getSelectionNames() if s.startswith(selection_prefix)]

print("{} zones with prefix {} have been found:".format(len(zones), selection_prefix))
print(", ".join(zones))

9 zones with prefix subdomain have been found:
subdomain_0, subdomain_1, subdomain_2, subdomain_3, subdomain_4, subdomain_5, subdomain_6, subdomain_7, subdomain_8


In [4]:
# create nodal representation of subdomain selections
cachefile = file_results+".nodecache"

if os.path.isfile(cachefile) and not force_cache_update:
    print("cache file found, reading from {}.".format(cachefile))
    f = open(cachefile)
    selection_nodes = json.load(f)
    f.close()
else:    
    # convert all selections to nodes
    print("no cache file found, creating one in {}...".format(cachefile))
    
    selection_nodes = {}
    for z in zones:
        selection_nodes[z] = doc.c.sel.convert(z, ifm.Enum.SEL_NODES)
        
    # Save to JSON for next time
    f = open(cachefile, "w")
    json.dump(selection_nodes, f)
    f.close()
    
    print("done.")

no cache file found, creating one in ./results/subdomain_test1.dac.nodecache...
done.


**Check for borders**

This is a precheck to determine how many nodes are shared between zones (zone combinations with no shared nodes will be skippted). You can use this information for troubleshooting.

In [5]:
# pre-check how many borders with overlapping nodes exist
df_borders = pd.DataFrame(index=zones, columns=zones)
for a in zones:
    for b in zones:
        df_borders.loc[a, b] = len(set(selection_nodes[a]) & set(selection_nodes[b]))
df_borders.to_excel(file_results+"_borders.xlsx")
df_borders

,subdomain_0,subdomain_1,subdomain_2,subdomain_3,subdomain_4,subdomain_5,subdomain_6,subdomain_7,subdomain_8
subdomain_0,242,22,0,22,2,0,0,0,0
subdomain_1,22,374,22,2,34,2,0,0,0
subdomain_2,0,22,154,0,2,14,0,0,0
subdomain_3,22,2,0,286,26,0,22,2,0
subdomain_4,2,34,2,26,442,26,2,34,2
subdomain_5,0,2,14,0,26,182,0,2,14
subdomain_6,0,0,0,22,2,0,242,22,0
subdomain_7,0,0,0,2,34,2,22,374,22
subdomain_8,0,0,0,0,2,14,0,22,154


*Table: Matrix with number of border nodes between zones*

**Compute the Interflux Matrix**

This is the actual computation. Note that it takes considerable time, as a separate matrix assembly is required by FEFLOW for each Balance zone.

In [6]:
def df_internal(doi, mds, corner_node_strategy="equal"):
    
    # parameter handling
    if corner_node_strategy != "equal":
        raise ValueError('"equal" is currently the only supported corner node strategy')

    # convert element selections to node selection, and intersect md with doi
    nodes_doi = selection_nodes[doi]
    nodes_mds = {s : set(selection_nodes[s]) & set(selection_nodes[doi]) for s in mds}

    # check if all elements are covered
    element_other = set(range(doc.getNumberOfElements())) - doc.c.sel.set(doi)
    for md in mds:
        element_other -= doc.c.sel.set(md)
    if len(element_other) > 0:
        raise RuntimeError("Not all elements covered!")

    # get budget, masked by itself
    sdba = doc.c.bdgt.get_subdomainbudgettransfer(doi, doi)

    # get nodal fluxes (border) 
    df_q = sdba.df_nodal_flux()
    
    # add membership flag: is node member of MD? (one column per MD) 
    for s in mds:
        df_q[s] = df_q.index.isin(nodes_mds[s])
        
    # count memberships
    df_q["n_memberships"] = df_q[mds].T.sum()

    # calculate nodal contribution to each md: total_flux div by number of memberships
    for s in mds:
        df_q["q_{}".format(s)] = 0.
        df_q.loc[df_q[s]  , "q_{}".format(s)] = df_q.q / df_q.n_memberships
        
    # create dataframe with fluxes
    df_result = pd.DataFrame(df_q[["q_{}".format(s) for s in mds]].sum(), columns=["net"])
    df_result.index = [s[2:] for s in df_result.index]
    
   
    return df_result

In [7]:
# create an empty data frame for results
df_fluxes = pd.DataFrame(index=zones)

# intiialize progress indicator
# run all combinations
for doi in zones:
    df_fluxes[doi] = df_internal(doi, zones)
     
    # progress
   
    #print("{} <- {} budget: {} m³/d\t[{} nodes]".format(a, b, df_fluxes.loc[a, b], len(set(selection_nodes[a]) & set(selection_nodes[b]))))

# mask diagonal
np.fill_diagonal(df_fluxes.values, np.nan) 

df_fluxes.to_excel(file_results+"_fluxes.xlsx")
df_fluxes

,subdomain_0,subdomain_1,subdomain_2,subdomain_3,subdomain_4,subdomain_5,subdomain_6,subdomain_7,subdomain_8
subdomain_0,NaN,0.105830,0.000000,-0.105753,-0.000041,0.000000,0.000000,0.000000,0.000000
subdomain_1,-0.100649,NaN,0.337564,-0.005139,-0.221582,-0.002808,0.000000,0.000000,0.000000
subdomain_2,0.000000,-0.325745,NaN,0.000000,-0.009010,-0.155667,0.000000,0.000000,0.000000
subdomain_3,0.100649,0.005144,0.000000,NaN,0.139551,0.000000,-0.250436,0.000243,0.000000
subdomain_4,0.000036,0.219916,0.009578,-0.136151,NaN,0.093427,-0.008823,-0.176776,-0.000997
subdomain_5,0.000000,0.002240,0.162436,0.000000,-0.100071,NaN,0.000000,-0.004178,-0.059034
subdomain_6,0.000000,0.000000,0.000000,0.241904,0.008289,0.000000,NaN,0.240984,0.000000
subdomain_7,0.000000,0.000000,0.000000,0.000291,0.182102,0.004203,-0.249564,NaN,0.059034
subdomain_8,0.000000,0.000000,0.000000,0.000000,0.000972,0.062240,0.000000,-0.064209,NaN


*Table: Matrix with Fluxes between zones*

**Error Check**

The folllowing table shows the relative error of the computation in percent. 
The absolute error is calculated by adding each entry of the matrix to its transposed element (corresponding to the sum of flux from any zone A to B and flux from zone B to A) and should theoretically equal zero. However, in many cases this will not be the case, for reasons explained elsewhere.

The error is normed by division by the element value (flux from A to B in above example) to put it into perspective.

In [8]:
# get relative error (percent)
df_error = (df_fluxes + df_fluxes.T) / df_fluxes * 100
df_error.to_excel(file_results+"_errors.xlsx")
df_error

,subdomain_0,subdomain_1,subdomain_2,subdomain_3,subdomain_4,subdomain_5,subdomain_6,subdomain_7,subdomain_8
subdomain_0,NaN,4.894790,NaN,4.825545,13.638756,NaN,NaN,NaN,NaN
subdomain_1,-5.146711,NaN,3.501008,-0.109656,0.752091,20.230376,NaN,NaN,NaN
subdomain_2,NaN,-3.628025,NaN,NaN,-6.305694,-4.348802,NaN,NaN,NaN
subdomain_3,-5.070211,0.109536,NaN,NaN,2.436275,NaN,3.406842,219.788413,NaN
subdomain_4,-15.792682,-0.757790,5.931661,-2.497111,NaN,-7.111170,6.047095,-3.013044,2.507988
subdomain_5,NaN,-25.361002,4.167563,NaN,6.639055,NaN,NaN,-0.598484,-5.430100
subdomain_6,NaN,NaN,NaN,-3.527001,-6.436305,NaN,NaN,-3.560391,NaN
subdomain_7,NaN,NaN,NaN,183.480528,2.924915,0.594924,3.437986,NaN,-8.765048
subdomain_8,NaN,NaN,NaN,NaN,-2.572506,5.150427,NaN,8.058699,NaN


*Table: Relative Errror of Fluxes between zones in percent*